In [1]:
import datetime
import numpy as np
import os 

from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l1

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

from local_vars import root_folder

num_classes = 2
ultrasound_size = 128

notebook_save_folder = r"LeaveOneOutNotebooks"
model_save_folder = r"LeaveOneOutModels"

train_data_folder = r"LeaveOneOutTrainArrays"
test_data_folder = r"LeaveOneOutTestArrays"
childrens_test_data_folder = r"ChildrensTestArrays"

# Augmentation parameters
max_rotation_angle = 10

# Model parameters
filter_multiplier = 10

# Learning parameters
num_epochs = 30
batch_size = 24
max_learning_rate = 0.002
min_learning_rate = 0.00001
regularization_rate = 0.0001
WCE_weights = np.array([0.01, 0.25])
learning_rate_decay = (max_learning_rate - min_learning_rate) / num_epochs

# Other parameters
num_show = 2
outList = [r"q000", r"q001", r"q002", r"q003", r"q004", r"q005", r"q006", r"q007"]

Using TensorFlow backend.


In [2]:
# Data Importing
def get_train_data(root_folder, train_data_folder, ultrasound_file, segmentation_file):
    train_data_fullpath = os.path.join(root_folder, train_data_folder)

    ultrasound_fullname = os.path.join(train_data_fullpath, ultrasound_file)
    segmentation_fullname = os.path.join(train_data_fullpath, segmentation_file)

    ultrasound_data = np.load(ultrasound_fullname)
    segmentation_data = np.load(segmentation_fullname)

    num_ultrasound = ultrasound_data.shape[0]
    num_segmentation = segmentation_data.shape[0]
    
    return ultrasound_data, segmentation_data, num_ultrasound, num_segmentation, train_data_fullpath

def get_test_data(root_folder, test_data_folder, test_ultrasound_file, test_segmentation_file):

    test_data_fullpath = os.path.join(root_folder, test_data_folder)

    test_ultrasound_fullname = os.path.join(test_data_fullpath, test_ultrasound_file)
    test_segmentation_fullname = os.path.join(test_data_fullpath, test_segmentation_file)

    test_ultrasound_data = np.load(test_ultrasound_fullname)
    test_segmentation_data = np.load(test_segmentation_fullname)

    num_test_ultrasound = test_ultrasound_data.shape[0]
    num_test_segmentation = test_segmentation_data.shape[0]
    
    return test_ultrasound_data, test_segmentation_data, num_test_ultrasound, num_test_segmentation, test_data_fullpath

In [3]:
# Create Ultrasound Segmentation Batch Generator Class
import keras.utils
import scipy.ndimage

class UltrasoundSegmentationBatchGenerator(keras.utils.Sequence):
    
    def __init__(self,
                 x_set,
                 y_set,
                 batch_size,
                 image_dimensions=(ultrasound_size, ultrasound_size),
                 shuffle=True,
                 n_channels=1,
                 n_classes=2):
        self.x = x_set
        self.y = y_set
        self.batch_size = batch_size
        self.image_dimensions = image_dimensions
        self.shuffle = shuffle
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.number_of_images = self.x.shape[0]
        self.indexes = np.arange(self.number_of_images)
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
    
    def __len__(self):
        return int(np.floor(self.number_of_images / self.batch_size))
    
    def on_epoch_end(self):
        self.indexes = np.arange(self.number_of_images)
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
    
    def __getitem__(self, index):
        batch_indexes = self.indexes[index*self.batch_size : (index+1)*self.batch_size]
        x = np.empty((self.batch_size, *self.image_dimensions, self.n_channels))
        y = np.empty((self.batch_size, *self.image_dimensions))
        
        for i in range(self.batch_size):
            flip_flag = np.random.randint(2)
            if flip_flag == 1:
                x[i,:,:,:] = np.flip(self.x[batch_indexes[i],:,:,:], axis=1)
                y[i,:,:] = np.flip(self.y[batch_indexes[i],:,:], axis=1)
            else:
                x[i,:,:,:] = self.x[batch_indexes[i],:,:,:]
                y[i,:,:] = self.y[batch_indexes[i],:,:]
        
        angle = np.random.randint(-max_rotation_angle, max_rotation_angle) 
        x_rot = scipy.ndimage.interpolation.rotate(x, angle, (1,2), False, mode="constant", cval=0, order=0)
        y_rot = scipy.ndimage.interpolation.rotate(y, angle, (1,2), False, mode="constant", cval=0, order=0)
        
        x_rot = np.clip(x_rot, 0.0, 1.0)
        y_rot = np.clip(y_rot, 0.0, 1.0)
        
        y_onehot = keras.utils.to_categorical(y_rot, self.n_classes)
        
        return x_rot, y_onehot

In [4]:
def dialateStack(segmentation_data, iterations):
    
    return np.array([scipy.ndimage.binary_dilation(y, iterations=iterations) for y in segmentation_data])

In [5]:
# U-Net Model Construction
from keras import backend as K

def nvidia_unet(patch_size=ultrasound_size, num_classes=num_classes, regularization_rate=0.):
    input_ = Input((patch_size, patch_size, 1))
    skips = []
    output = input_
    
    num_layers = int(np.floor(np.log2(patch_size)))
    down_conv_kernel_sizes = np.zeros([num_layers], dtype=int)
    down_filter_numbers = np.zeros([num_layers], dtype=int)
    up_conv_kernel_sizes = np.zeros([num_layers], dtype=int)
    up_filter_numbers = np.zeros([num_layers], dtype=int)
    
    for layer_index in range(num_layers):
        down_conv_kernel_sizes[layer_index] = int(3)
        down_filter_numbers[layer_index] = int( (layer_index + 1) * filter_multiplier + num_classes )
        up_conv_kernel_sizes[layer_index] = int(4)
        up_filter_numbers[layer_index] = int( (num_layers - layer_index - 1) * filter_multiplier + num_classes )
    
    for shape, filters in zip(down_conv_kernel_sizes, down_filter_numbers):
        skips.append(output)
        output = Conv2D(filters, (shape, shape), strides=2, padding="same", activation="relu", bias_regularizer=l1(regularization_rate))(output)
    
    for shape, filters in zip(up_conv_kernel_sizes, up_filter_numbers):
        output = keras.layers.UpSampling2D()(output)
        skip_output = skips.pop()
        output = concatenate([output, skip_output], axis=3)
        if filters != num_classes:
            activation = "relu"
            output = Conv2D(filters, (shape, shape), activation="relu", padding="same", bias_regularizer=l1(regularization_rate))(output)
            output = BatchNormalization(momentum=.9)(output)
        else:
            activation = "softmax"
            output = Conv2D(filters, (shape, shape), activation="softmax", padding="same", bias_regularizer=l1(regularization_rate))(output)
    
    assert len(skips) == 0
    return Model([input_], [output])

def dice_coef(y_true, y_pred, smooth=1):
    """
    Dice = (2*|X & Y|)/ (|X|+ |Y|)
         =  2*sum(|A*B|)/(sum(A^2)+sum(B^2))
    ref: https://arxiv.org/pdf/1606.04797v1.pdf
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    
    return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)

def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    """
    weights = K.variable(weights)
        
    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        
        return loss
    
    return loss

In [6]:
for out in outList:
    #
    # Add in Children's Predictions at the same time (so we're using the same model for both)
    #
    ultrasound_file = out + r"out_ultrasound.npy"
    segmentation_file = out + r"out_segmentation.npy"

    # Get Leave Out One Filenames
    test_ultrasound_file = out + r"_ultrasound.npy"
    test_segmentation_file = out + r"_segmentation.npy"
    test_prediction_file = out + r"_prediction.npy"
    
    # Get Childrens Test Filenames
    childrens_test_ultrasound_file = r"ultrasound-test.npy"
    childrens_test_segmentation_file = r"segmentation-test.npy"
    childrens_test_prediction_file = out + r"_prediction.npy"
    
    print("#############################################")
    print("Training network that leaves out", out)
    print("#############################################\n")
    
    # Get Data
    # Train
    ultrasound_data, segmentation_data, num_ultrasound, num_segmentation, train_data_fullpath = get_train_data(root_folder, train_data_folder, ultrasound_file, segmentation_file)
    # L-1-O Test
    test_ultrasound_data, test_segmentation_data, num_test_ultrasound, num_test_segmentation, test_data_fullpath = get_test_data(root_folder, test_data_folder, test_ultrasound_file, test_segmentation_file)
    # Childrens Test
    childrens_test_ultrasound_data, childrens_test_segmentation_data, childrens_num_test_ultrasound, childrens_num_test_segmentation, childrens_test_data_fullpath = get_test_data(root_folder, childrens_test_data_folder, childrens_test_ultrasound_file, childrens_test_segmentation_file)
    
    # Prepare Dilated Outputs
    width = 1
    segmentation_dilated = dialateStack(segmentation_data[:, :, :, 0], width)
    segmentation_dilated[:, :, :] = segmentation_data[:, :, :, 0]
    
    # Create Model
    model = nvidia_unet(ultrasound_size, num_classes, regularization_rate)
    model.compile(optimizer=keras.optimizers.adam(lr=max_learning_rate, decay=learning_rate_decay),
              loss=[weighted_categorical_crossentropy(WCE_weights)],
              metrics=["accuracy", dice_coef])
    
    # Create Train and Test Generators
    # Train
    training_generator = UltrasoundSegmentationBatchGenerator(ultrasound_data, segmentation_dilated, batch_size)
    # L-1-0 Test
    test_generator = UltrasoundSegmentationBatchGenerator(test_ultrasound_data, test_segmentation_data[:, :, :, 0], batch_size)
    # Childrens Test
    childrens_test_generator = UltrasoundSegmentationBatchGenerator(childrens_test_ultrasound_data, childrens_test_segmentation_data[:, :, :, 0], batch_size)
    
    training_time_start = datetime.datetime.now()
    
    # Validate on ONLY L-1-O Test Data
    training_log = model.fit_generator(training_generator,
                                       validation_data=test_generator,
                                       epochs=num_epochs,
                                       verbose=0)
        
    training_time_stop = datetime.datetime.now()
    
    print("val_acc:", training_log.history['val_acc'][-1], "val loss:", training_log.history['val_loss'][-1], "val_dice:", training_log.history['val_dice_coef'][-1])
    print("\nTraining started at: {}".format(training_time_start))
    print("Training stopped at: {}".format(training_time_stop))
    print("Total training time: {}\n".format(training_time_stop-training_time_start))
    
    # Predict on Test Data
    y_pred = model.predict(test_ultrasound_data)
    childrens_y_pred = model.predict(childrens_test_ultrasound_data)
    
    # Saving L-1-O prediction for further evaluation
    test_prediction_fullname = os.path.join(test_data_fullpath, test_prediction_file)
    np.save(test_prediction_fullname, y_pred)
    print("Predictions saved to: {}\n".format(test_prediction_fullname))

    # Saving L-1-O prediction for further evaluation
    childrens_test_prediction_fullname = os.path.join(childrens_test_data_fullpath, childrens_test_prediction_file)
    np.save(childrens_test_prediction_fullname, childrens_y_pred)
    print("Childrens predictions saved to: {}\n".format(childrens_test_prediction_fullname))
    
    # Archive model and notebook with unique filenames based on timestamps
    timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    saved_models_fullpath = os.path.join(root_folder, model_save_folder)
    if not os.path.exists(saved_models_fullpath):
        os.makedirs(saved_models_fullpath)
        print("Creating folder: {}".format(saved_models_fullpath))
    model_file_name = "model_" + timestamp + ".h5"
    model_fullname = os.path.join(saved_models_fullpath, model_file_name)
    model.save(model_fullname)
    print("Model saved to: {}\n".format(model_fullname))

#############################################
Training network that leaves out q000
#############################################

val_acc: 0.9861342367671785 val loss: 0.0004754458190984137 val_dice: 0.9925244138354347

Training started at: 2019-10-11 07:28:30.213931
Training stopped at: 2019-10-11 07:33:21.688138
Total training time: 0:04:51.474207

Predictions saved to: c:\Data\LeaveOneOutTestArrays\q000_prediction.npy

Childrens predictions saved to: c:\Data\ChildrensTestArrays\q000_prediction.npy

Model saved to: c:\Data\LeaveOneOutModels\model_2019-10-11_07-33-24.h5

#############################################
Training network that leaves out q001
#############################################

val_acc: 0.9955722221306392 val loss: 0.0011215169943170622 val_dice: 0.9974862464836666

Training started at: 2019-10-11 07:33:28.442798
Training stopped at: 2019-10-11 07:38:16.180118
Total training time: 0:04:47.737320

Predictions saved to: c:\Data\LeaveOneOutTestArrays\q001_predictio

In [7]:
# Save HTML copy of full Notebook.
timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
saved_notebooks_fullpath = os.path.join(root_folder, notebook_save_folder)
if not os.path.exists(saved_notebooks_fullpath):
    os.makedirs(saved_notebooks_fullpath)
    print("Creating folder: {}".format(saved_notebooks_fullpath))

notebook_file_name = "notebook_" + timestamp + ".html"
notebook_fullname = os.path.join(saved_notebooks_fullpath, notebook_file_name)
import time
time.sleep(30)
os.system("jupyter nbconvert --to html Segmentation2-QueensToChildrens --output " + notebook_fullname)
print("Notebook saved to: {}".format(notebook_fullname))

Notebook saved to: c:\Data\LeaveOneOutNotebooks\notebook_2019-10-11_08-10-53.html
